In [70]:
import numpy as np #linear algebra
import pandas as pd #data manipulation
import datetime as dt 
import time
import warnings

# Data Extraction

In [71]:
# read in data
df = pd.read_csv('customer_segmentation_data_input.csv', dtype={'CustomerID': str}, encoding="ISO-8859-1") 
df.info()
print('\n')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']) #convert to datetime object for proper comparison
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  object 
 7   Country      541909 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 33.1+ MB




,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


# Data Preprocessing

In [72]:
#Columns needed to be preprocessed are: Quantity, InvoiceDate, CustomerID, and Country

In [73]:
#Quantity

df_current = df[df['Quantity'] > 0] #only include columns where quantity is positive, remove returns
print(len(df_current))

531285


In [74]:
#Country

# find country that occurs most frequently 
country_counts = df['Country'].value_counts()
most_frequent_country = country_counts.idxmax()
print(most_frequent_country)

# country that occurs most frequently is United Kingdom

# only keep entries whose country is the United Kingdom
df_current = df_current[df_current['Country'] == 'United Kingdom']
print(len(df_current))

United Kingdom
486286


In [75]:
#InvoiceDate

# find the most recent, that is, last invoice date in data set
last_invoice_date = df_current['InvoiceDate'].max()
print(last_invoice_date)

# last invoice date is 2011-12-09

# only keep entries from the last year: 2010-12-09 to 2011-12-09
df_current = df_current[df_current['InvoiceDate'] >= pd.to_datetime('2010-12-09')]
print(len(df_current))

2011-12-09 12:49:00
467857


In [76]:
#CustomerID

#dropna() removes rows with missing values, and takes in three parameters
  #parameter 1: specifies subset of columns to consider for missing values
  #parameter 2: specifies condition for dropping rows
  #parameter 3: specifies whether to modify current dataframe or make new one

# consider CustomerID columns, drop row if CustomerID column is missing, and modify df_current itself
df_current.dropna(subset=['CustomerID'], how='all', inplace=True) 
print(len(df_current))

342478


In [77]:
# print results from preprocessing
print('\n')
print("Top 5 Entries in Preprocessed Dataset:")
display(df_current.head())
print('\n')
print("Bottom 5 Entries in Preprocessed Dataset:")
display(df_current.tail())
print('\n')
print("Preprocessed Dataset Check")
print("The quantities in the preprocessed dataset are nonnegative: ", (df_current['Quantity'] > 0).all()) 
print("The country in the preprocessed dataset is just the United Kingdom: ", (df_current['Country'] == 'United Kingdom').all())
print("The entries in the preprocessed dataset have start date of", (df_current['InvoiceDate'].min()), "and end date of", (df_current['InvoiceDate'].max()))
print("There are no entries in the preprocessed dataset with blank customer IDs: ", not df_current['CustomerID'].isnull().any())
print('\n')
print("Preprocessed Dataset Summary")
print("The number of entries in the preprocessed dataset are: ", len(df_current))
print("The number of transactions in the preprocessed dataset are: ", df_current['InvoiceNo'].nunique())
print("The number of customers in the preprocessed dataset are: ", df_current['CustomerID'].nunique() )



Top 5 Entries in Preprocessed Dataset:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
19632,537879,22114,HOT WATER BOTTLE TEA AND SYMPATHY,12,2010-12-09 08:34:00,3.95,14243,United Kingdom
19633,537879,22835,HOT WATER BOTTLE I AM SO POORLY,8,2010-12-09 08:34:00,4.65,14243,United Kingdom
19634,537879,85150,LADIES & GENTLEMEN METAL SIGN,6,2010-12-09 08:34:00,2.55,14243,United Kingdom
19635,537879,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,4,2010-12-09 08:34:00,7.95,14243,United Kingdom
19636,537879,21524,DOORMAT SPOTTY HOME SWEET HOME,2,2010-12-09 08:34:00,7.95,14243,United Kingdom




Bottom 5 Entries in Preprocessed Dataset:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541889,581585,22466,FAIRY TALE COTTAGE NIGHT LIGHT,12,2011-12-09 12:31:00,1.95,15804,United Kingdom
541890,581586,22061,LARGE CAKE STAND HANGING STRAWBERY,8,2011-12-09 12:49:00,2.95,13113,United Kingdom
541891,581586,23275,SET OF 3 HANGING OWLS OLLIE BEAK,24,2011-12-09 12:49:00,1.25,13113,United Kingdom
541892,581586,21217,RED RETROSPOT ROUND CAKE TINS,24,2011-12-09 12:49:00,8.95,13113,United Kingdom
541893,581586,20685,DOORMAT RED RETROSPOT,10,2011-12-09 12:49:00,7.08,13113,United Kingdom




Preprocessed Dataset Check
The quantities in the preprocessed dataset are nonnegative:  True
The country in the preprocessed dataset is just the United Kingdom:  True
The entries in the preprocessed dataset have start date of 2010-12-09 08:34:00 and end date of 2011-12-09 12:49:00
There are no entries in the preprocessed dataset with blank customer IDs:  True


Preprocessed Dataset Summary
The number of entries in the preprocessed dataset are:  342478
The number of transactions in the preprocessed dataset are:  16017
The number of customers in the preprocessed dataset are:  3863


# Data Analysis: RFM

## Recency

In [78]:
# Objective: calculate the recency for each customer, which is the number of days since the customer last made a purchase

In [79]:
# create a column with strictly invoice dates, no times
df_current['StrictInvoiceDate'] = (pd.DatetimeIndex(df_current['InvoiceDate'])).date #only extract the date from the InvoiceDate column, and store in new column called StrictInvoiceDate
df_current.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,StrictInvoiceDate
19632,537879,22114,HOT WATER BOTTLE TEA AND SYMPATHY,12,2010-12-09 08:34:00,3.95,14243,United Kingdom,2010-12-09
19633,537879,22835,HOT WATER BOTTLE I AM SO POORLY,8,2010-12-09 08:34:00,4.65,14243,United Kingdom,2010-12-09
19634,537879,85150,LADIES & GENTLEMEN METAL SIGN,6,2010-12-09 08:34:00,2.55,14243,United Kingdom,2010-12-09
19635,537879,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,4,2010-12-09 08:34:00,7.95,14243,United Kingdom,2010-12-09
19636,537879,21524,DOORMAT SPOTTY HOME SWEET HOME,2,2010-12-09 08:34:00,7.95,14243,United Kingdom,2010-12-09


In [80]:
# create a separate table with the customer ID and their correponding last purchase date

# group by customer ID, and for each customer, take the max of the StrictInvoiceDate's, which is the most recent purchase date
df_recency = df_current.groupby(by = 'CustomerID', as_index = False)['StrictInvoiceDate'].max() #new table called df_recency
df_recency.rename(columns={df_recency.columns[1]: 'MostRecentPurchaseDate'}, inplace = True) #update column name
df_recency.head()

,CustomerID,MostRecentPurchaseDate
0,12346,2011-01-18
1,12747,2011-12-07
2,12748,2011-12-09
3,12749,2011-12-06
4,12820,2011-12-06


In [81]:
# calculate the end date
endDate = df_current['InvoiceDate'].max() 
endDate = pd.to_datetime(endDate) #converts endDate to pandas.Timestamp object
print(endDate)

2011-12-09 12:49:00


In [82]:
# calculate the recency

# recency value for customer = end date - most recent purchase date for that customer
df_recency['MostRecentPurchaseDate'] = pd.to_datetime(df_recency['MostRecentPurchaseDate']) #converts all entries in column to pandas.Timestamp objects
df_recency['Recency'] = (endDate - df_recency['MostRecentPurchaseDate']).dt.days #subtraction between two pandas.Timestamp objects
df_recency.head()

,CustomerID,MostRecentPurchaseDate,Recency
0,12346,2011-01-18,325
1,12747,2011-12-07,2
2,12748,2011-12-09,0
3,12749,2011-12-06,3
4,12820,2011-12-06,3


In [83]:
# remove StrictInvoiceDate from df_current
df_current.drop(columns=['StrictInvoiceDate'], inplace = True)
df_current.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
19632,537879,22114,HOT WATER BOTTLE TEA AND SYMPATHY,12,2010-12-09 08:34:00,3.95,14243,United Kingdom
19633,537879,22835,HOT WATER BOTTLE I AM SO POORLY,8,2010-12-09 08:34:00,4.65,14243,United Kingdom
19634,537879,85150,LADIES & GENTLEMEN METAL SIGN,6,2010-12-09 08:34:00,2.55,14243,United Kingdom
19635,537879,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,4,2010-12-09 08:34:00,7.95,14243,United Kingdom
19636,537879,21524,DOORMAT SPOTTY HOME SWEET HOME,2,2010-12-09 08:34:00,7.95,14243,United Kingdom


In [84]:
# remove MostRecentPurchaseDate from df_recency
df_recency.drop(columns=['MostRecentPurchaseDate'], inplace = True)
df_recency.head()

,CustomerID,Recency
0,12346,325
1,12747,2
2,12748,0
3,12749,3
4,12820,3


## Frequency

In [85]:
# Objective: calculate the frequency for each customer, which is the number of times a customer has made a purchase

In [86]:
#for each of the customers, calculate their total number of invoices, no duplicates

# group by customer IDs, and for each customer, calculate the number of unique entries in InvoiceNo
df_frequency = df_current.groupby(by=['CustomerID'], as_index = False).agg({'InvoiceNo': 'nunique'})
df_frequency.rename(columns={df_frequency.columns[1]: 'Frequency'}, inplace = True) #update column name
df_frequency.head()

,CustomerID,Frequency
0,12346,1
1,12747,10
2,12748,196
3,12749,5
4,12820,4


## Monetary

In [87]:
# Objective: calculate the monetary value for each customer, which amount of money the customer has spent in total

In [88]:
# monetary value per entry = unit price * quantity of that entry
df_current['TotalPrice'] = df_current['Quantity'] * df_current['UnitPrice']
df_current.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
19632,537879,22114,HOT WATER BOTTLE TEA AND SYMPATHY,12,2010-12-09 08:34:00,3.95,14243,United Kingdom,47.4
19633,537879,22835,HOT WATER BOTTLE I AM SO POORLY,8,2010-12-09 08:34:00,4.65,14243,United Kingdom,37.2
19634,537879,85150,LADIES & GENTLEMEN METAL SIGN,6,2010-12-09 08:34:00,2.55,14243,United Kingdom,15.3
19635,537879,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,4,2010-12-09 08:34:00,7.95,14243,United Kingdom,31.8
19636,537879,21524,DOORMAT SPOTTY HOME SWEET HOME,2,2010-12-09 08:34:00,7.95,14243,United Kingdom,15.9


In [89]:
# for each of the customers, calculate their total monetary expenditure

# group by customer IDs, and for each customer, calculate the sum of the TotalPrice, which is the sum of the total price per entry
df_monetary = df_current.groupby(by = 'CustomerID', as_index = False).agg({'TotalPrice': 'sum'})
df_monetary.rename(columns={df_monetary.columns[1]: 'Monetary'}, inplace = True) #update column name
df_monetary.head()

,CustomerID,Monetary
0,12346,77183.60
1,12747,3837.45
2,12748,31854.58
3,12749,4090.88
4,12820,942.34


In [90]:
# remove MostRecentPurchaseDate from df_recency
df_current.drop(columns=['TotalPrice'], inplace = True)
df_current.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
19632,537879,22114,HOT WATER BOTTLE TEA AND SYMPATHY,12,2010-12-09 08:34:00,3.95,14243,United Kingdom
19633,537879,22835,HOT WATER BOTTLE I AM SO POORLY,8,2010-12-09 08:34:00,4.65,14243,United Kingdom
19634,537879,85150,LADIES & GENTLEMEN METAL SIGN,6,2010-12-09 08:34:00,2.55,14243,United Kingdom
19635,537879,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,4,2010-12-09 08:34:00,7.95,14243,United Kingdom
19636,537879,21524,DOORMAT SPOTTY HOME SWEET HOME,2,2010-12-09 08:34:00,7.95,14243,United Kingdom


## RFM Concatenation

In [91]:
# create a table with the customer IDs, and their corresponding recency, frequency, and monetary values
df_rfm = pd.concat([df_recency, df_frequency, df_monetary],  axis=1) # axis = 1 so tables concatenated side-by-side
df_rfm = df_rfm.T.drop_duplicates().T # customerID columns are duplicates, drop them
df_rfm.set_index('CustomerID',inplace=True) # customerID column becomes the index
df_rfm['Recency'] = pd.to_numeric(df_rfm['Recency'], errors='coerce')  # convert to numeric
df_rfm['Frequency'] = pd.to_numeric(df_rfm['Frequency'], errors='coerce') # convert to numeric
df_rfm['Monetary'] = pd.to_numeric(df_rfm['Monetary'], errors='coerce') # convert to numeric
df_rfm.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346,325,1,77183.60
12747,2,10,3837.45
12748,0,196,31854.58
12749,3,5,4090.88
12820,3,4,942.34


# Customer Segmentation

In [92]:
# split recency, frequency, and monetary values into quintiles 
quintiles = df_rfm.quantile(q = [0.2, 0.4, 0.6, 0.8, 1.0])
quintiles # quintiles table shows, for example, that any customer with a recency value of 12.0 is in the 20th percentile

,Recency,Frequency,Monetary
0.2,12.0,1.0,237.186
0.4,31.0,2.0,458.438
0.6,68.0,3.0,890.300
0.8,171.0,5.0,1906.644
1.0,365.0,196.0,233736.930


In [93]:
# classify each of the recency, frequency, and monetary values in their respective bins based on the quintile they are in
df_rfm['RecencyBin'] = pd.qcut(df_rfm['Recency'].rank(method='first'), 5, labels = [5, 4, 3, 2, 1]) # low recency has bin value of 5
df_rfm['FrequencyBin']= pd.qcut(df_rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5]) # high frequency has bin value of 5
df_rfm['MonetaryBin'] = pd.qcut(df_rfm['Monetary'].rank(method='first'), 5, labels = [1, 2, 3, 4, 5]) # high monetary has bin value of 5
df_rfm.head()

,Recency,Frequency,Monetary,RecencyBin,FrequencyBin,MonetaryBin
CustomerID,,,,,,
12346,325,1,77183.60,1,1,5
12747,2,10,3837.45,5,5,5
12748,0,196,31854.58,5,5,5
12749,3,5,4090.88,5,4,5
12820,3,4,942.34,5,4,4


In [94]:
# get RFM Value

# convert all entries to string
df_rfm['RecencyBin'] = df_rfm['RecencyBin'].astype(str)
df_rfm['FrequencyBin'] = df_rfm['FrequencyBin'].astype(str)
df_rfm['MonetaryBin'] = df_rfm['MonetaryBin'].astype(str)

# concatenate strings to get RFM value
df_rfm['RFM'] = df_rfm['RecencyBin'] + df_rfm['FrequencyBin'] + df_rfm['MonetaryBin']

df_rfm.head()

,Recency,Frequency,Monetary,RecencyBin,FrequencyBin,MonetaryBin,RFM
CustomerID,,,,,,,
12346,325,1,77183.60,1,1,5,115
12747,2,10,3837.45,5,5,5,555
12748,0,196,31854.58,5,5,5,555
12749,3,5,4090.88,5,4,5,545
12820,3,4,942.34,5,4,4,544


In [95]:
print('\n')
print("Snapshot of Best Customers:")
display(df_rfm[df_rfm["RFM"]=="555"].head())
print('\n')
print("Snapshot of Worst Customers:")
df_rfm[df_rfm["RFM"]=="111"].head()

# those with low recency, high frequency, and high monetary values have a high RFM value of 555
# those with high recency, low frequency, and low monetary values have a low RFM value of 111



Snapshot of Best Customers:


,Recency,Frequency,Monetary,RecencyBin,FrequencyBin,MonetaryBin,RFM
CustomerID,,,,,,,
12747,2,10,3837.45,5,5,5,555
12748,0,196,31854.58,5,5,5,555
12839,2,13,5188.81,5,5,5,555
12841,4,23,3516.36,5,5,5,555
12856,7,6,2179.93,5,5,5,555




Snapshot of Worst Customers:


,Recency,Frequency,Monetary,RecencyBin,FrequencyBin,MonetaryBin,RFM
CustomerID,,,,,,,
12821,214,1,92.72,1,1,1,111
12831,262,1,215.05,1,1,1,111
12837,173,1,134.10,1,1,1,111
12902,264,1,138.68,1,1,1,111
12929,311,1,117.85,1,1,1,111


# Customer Segmentation

In [96]:
# Customers will be classified into different segments based on the following sources
# Source: https://connectif.ai/en/blog/what-are-rfm-scores-and-how-to-calculate-them/
# Source: https://www.linkedin.com/pulse/rfm-customer-segmentation-analysis-action-jason-tragakis-0ffrf/

In [97]:
# define categories and their corresponding RFM values

champions = ['555', '554', '544', '545', '454', '455', '445']
loyal_customers = ['543', '444', '435', '355', '354', '345', '344', '335']
potential_loyalist = ['553', '551', '552', '541', '542', '533', '532', '531', '452', '451', '442', '441', 
                      '431', '453', '433', '432', '423', '353', '352', '351', '342', '341', '333', '323']
new_customer = ['512', '511', '422', '421', '412', '411', '311']
promising = ['525', '524', '523', '522', '521', '515', '514', '513', '425', '424', '413', '414', '415', '315', '314', '313']
need_attention = ['535', '534', '443', '434', '343', '334', '325', '324']
about_to_sleep = ['331', '321', '312', '221', '213', '231', '241', '251']
at_risk = ['255', '254', '245', '244', '253', '252', '243', '242', '235', '234', '225',
           '224', '153', '152', '145', '143', '142', '135', '134', '133', '125', '124']
cannot_lose = ['155', '154', '144', '214', '215', '115', '114', '113']
hibernating = ['332', '322', '231', '241', '251', '233', '232', '223', '222', '132', '123', '122', '212', '211']
lost = ['111', '112', '121', '131', '141', '151']

# define function to categorize the input, which is the RFM value 

def categorize_rfm(rfm):
    if rfm in champions:
        return 'Champions'
    elif rfm in loyal_customers:
        return 'Loyal Customers'
    elif rfm in potential_loyalist:
        return 'Potential Loyalist'
    elif rfm in new_customer:
        return 'New Customer'
    elif rfm in promising:
        return 'Promising'
    elif rfm in need_attention:
        return 'Need Attention'
    elif rfm in about_to_sleep:
        return 'About To Sleep'
    elif rfm in at_risk:
        return 'At Risk'
    elif rfm in cannot_lose:
        return 'Cannot Lose'
    elif rfm in hibernating:
        return 'Hibernating'
    elif rfm in lost:
        return 'Lost'

# apply function to the RFM column using the apply() function
df_rfm['Segment'] = df_rfm['RFM'].apply(categorize_rfm)

display(df_rfm.head())
display(df_rfm.tail())

,Recency,Frequency,Monetary,RecencyBin,FrequencyBin,MonetaryBin,RFM,Segment
CustomerID,,,,,,,,
12346,325,1,77183.60,1,1,5,115,Cannot Lose
12747,2,10,3837.45,5,5,5,555,Champions
12748,0,196,31854.58,5,5,5,555,Champions
12749,3,5,4090.88,5,4,5,545,Champions
12820,3,4,942.34,5,4,4,544,Champions


,Recency,Frequency,Monetary,RecencyBin,FrequencyBin,MonetaryBin,RFM,Segment
CustomerID,,,,,,,,
18280,277,1,180.60,1,2,1,121,Lost
18281,180,1,80.82,1,2,1,121,Lost
18282,7,2,178.05,5,3,1,531,Potential Loyalist
18283,3,16,2094.88,5,5,5,555,Champions
18287,42,3,1837.28,3,4,4,344,Loyal Customers


In [98]:
# Generate summary results

# determine the number of entries per segment and store in dataframe
df_counts = df_rfm.groupby('Segment').size() # creates a Pandas series of the counts
df_counts = df_counts.reset_index(name='Count') # converts the Panda series to a dataframe, and renames column to count

# determine the mean and median for each of recency, frequency, and monetary and store in dataframe
df_summary = df_rfm[['Segment', 'Recency', 'Frequency', 'Monetary']].groupby('Segment').agg(['mean', 'median'])

# merge df_counts and df_summary on 'Segment' column so that 'Counts' is in df_summary

# flatten df_summary to single-level: cannot directly merge df_counts (single-level df) and df_summary (multi-level df) 
df_summary.columns = ['_'.join(col).rstrip('_') for col in df_summary.columns.values] # flatten df_summary to single-level df
df_summary = pd.merge(df_counts, df_summary, on='Segment', how='left') #merge df_counts and df_summary and store result in df_summary

# unflatten df_summary back to multilevel
indexes_to_unflatten = [2, 3, 4, 5, 6, 7] # indices of columns in single-level df_summary to unflatten
unflattened_columns = []
for i, col in enumerate(df_summary.columns): # iterate through each column
    if i in indexes_to_unflatten:
        # column's index is in the list; unflatten it
        unflattened_columns.append(tuple(col.split('_'))) # split on _ to get first-level and second-level column names
    else:
        # column index is not in the last, don't unflatten it, keep as is
        unflattened_columns.append((col, ''))  # first-level column name is itself, second-level column name is blank
df_summary.columns = pd.MultiIndex.from_tuples(unflattened_columns) # unflatten the columns
df_summary.set_index('Segment', inplace = True) # set 'Segment' as the index
df_summary = df_summary.round(2)
df_summary 

Count Recency        Frequency        Monetary         
                            mean median      mean median     mean   median
Segment                                                                   
About To Sleep       244   81.14   65.0      1.30    1.0   263.60   206.58
At Risk              385  133.83  112.0      3.69    3.0  1667.98  1128.91
Cannot Lose           85  231.39  219.0      2.09    1.0  2034.43   922.10
Champions            740   10.27    8.0     11.61    8.0  5767.55  2791.31
Hibernating          630  141.33  120.5      1.52    1.0   388.11   353.99
Lost                 427  259.48  261.0      1.08    1.0   189.47   170.75
Loyal Customers      375   35.80   36.0      5.30    5.0  2253.55  1575.16
Need Attention       251   31.18   29.0      3.08    3.0  1657.52   887.11
New Customer         251   26.58   24.0      1.06    1.0   209.85   191.32
Potential Loyalist   368   24.87   22.0      2.50    2.0   526.70   538.64
Promising            107   24.83   16.0      1.29    1.0   728.48   598.57

# Download Data

In [99]:
df_rfm.head()

,Recency,Frequency,Monetary,RecencyBin,FrequencyBin,MonetaryBin,RFM,Segment
CustomerID,,,,,,,,
12346,325,1,77183.60,1,1,5,115,Cannot Lose
12747,2,10,3837.45,5,5,5,555,Champions
12748,0,196,31854.58,5,5,5,555,Champions
12749,3,5,4090.88,5,4,5,545,Champions
12820,3,4,942.34,5,4,4,544,Champions


In [100]:
# create a copy to make slight modifications for download
df_rfm_download = df_rfm
df_rfm_download.reset_index(inplace=True) # customerID was index, now it is reverted back to a column

In [101]:
# adjust data types of the columns
df_rfm_download['CustomerID'] = df_rfm_download['CustomerID'].astype(str)
df_rfm['RecencyBin'] = pd.to_numeric(df_rfm['RecencyBin'], errors='coerce') 
df_rfm['FrequencyBin'] = pd.to_numeric(df_rfm['FrequencyBin'], errors='coerce')
df_rfm['MonetaryBin'] = pd.to_numeric(df_rfm['MonetaryBin'], errors='coerce')
df_rfm['RFM'] = pd.to_numeric(df_rfm['RFM'], errors='coerce')

# adjust monetary values to round to whole number
df_rfm['Monetary'] = df_rfm['Monetary'].astype(int)

#add new column with constant of 1 since there is one customer corresponding to one customer ID, and each column is one customer ID
# there are no duplicate customer IDs in df_rfm_download
df_rfm_download.insert(1, 'Customer', 1)
display(df_rfm_download.head())

df_rfm_download.dtypes

,CustomerID,Customer,Recency,Frequency,Monetary,RecencyBin,FrequencyBin,MonetaryBin,RFM,Segment
0,12346,1,325,1,77183,1,1,5,115,Cannot Lose
1,12747,1,2,10,3837,5,5,5,555,Champions
2,12748,1,0,196,31854,5,5,5,555,Champions
3,12749,1,3,5,4090,5,4,5,545,Champions
4,12820,1,3,4,942,5,4,4,544,Champions


CustomerID      object
Customer         int64
Recency          int64
Frequency        int64
Monetary         int64
RecencyBin       int64
FrequencyBin     int64
MonetaryBin      int64
RFM              int64
Segment         object
dtype: object

In [102]:
df_rfm_download.to_csv('customer_segmentation_rfm_output.csv', index = False) #downloads to your working directory

In [103]:
print(len(df_rfm_download[df_rfm_download['Frequency'] == 1]))
print(df_rfm_download['Frequency'].max())
print(len(df_rfm_download))

1357
196
3863
